# SVD
- 应用
    - 隐性语义索引（Latent Semantic Indexing,LSI）或 隐性语义分析（Latent Semantic Analysis,LSA）
        - 利用SVD构建出多个奇异值，代表文档的概念或潜在语义，概念
    - 推荐系统
        - 利用SVD从数据中构建一个主题空间，然后再该空间下计算相似度
        
- 矩阵分解
    - 不同的矩阵分解技术有不同的性质，其中有些更适合与某个应用，其中一种最常见的技术就是SVD
    - SVD可以表示为：$Data_{m\times n} = U_{m \times n} \Sigma_{m\times n} V^T_{m\times n}$ 

In [1]:
import numpy as np
U,Sigma,VT = np.linalg.svd([[1,1],[7,7]])
print(U,'\n')
# 这里的sigma实际上是矩阵，但是由于是对角阵为了方便保存，因此以向量的形式返回
print(Sigma,'\n')
print(VT,'\n')

[[-0.14142136 -0.98994949]
 [-0.98994949  0.14142136]] 

[1.00000000e+01 2.82797782e-16] 

[[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]] 



In [2]:
def loadExData():
    return[[1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [1, 1, 1, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1]]

Data = loadExData()
U,Sigma,VT = np.linalg.svd(Data)
Sigma

array([9.72140007e+00, 5.29397912e+00, 6.84226362e-01, 4.11502614e-16,
       1.36030206e-16])

In [3]:
# 重构原始矩阵
Sig3 = np.mat([[Sigma[0],0,0],[0,Sigma[1],0],[0,0,Sigma[2]]])
Sig3

matrix([[9.72140007, 0.        , 0.        ],
        [0.        , 5.29397912, 0.        ],
        [0.        , 0.        , 0.68422636]])

In [4]:
U[:,:3]*Sig3*VT[:3,:]

matrix([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          7.75989921e-16,  7.71587483e-16],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          3.00514919e-16,  2.77832253e-16],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          2.18975112e-16,  2.07633779e-16],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
          3.00675663e-17, -1.28697294e-17],
        [ 1.00000000e+00,  1.00000000e+00, -5.48397422e-16,
          2.00000000e+00,  2.00000000e+00],
        [ 3.21319929e-16,  4.43562065e-16, -3.48967188e-16,
          3.00000000e+00,  3.00000000e+00],
        [ 9.71445147e-17,  1.45716772e-16, -1.52655666e-16,
          1.00000000e+00,  1.00000000e+00]])

如何知道保留几个奇异值呢？确定要保留的奇异值有许多启发策略，其中一个典型做法就是保留矩阵90%的信息，我们将所有奇异值平方求和，并累计到90%

# 基于协同过滤的推荐引擎
- 协同过滤是通过将用户和其他用户数据对比实现推荐
## 相似度计算
- 不利用专家给出的重要属性，而是利用用户对他们的意见来计算相似度，这就是协同过滤所使用的方法

1. 可以利用欧氏距离计算物品评分之间的相似度
2. Pearson correlation计算向量间相似度
3. 余弦相似度

In [5]:
import numpy as np

def ecludSim(inA,inB):
    return 1.0/(1.0 + np.linalg.norm(inA-inB))

def pearsSim(inA,inB):
    if len(inA)<3:
        return 1.0
    return 0.5+0.5*np.corrcoef(inA,inB,rowvar=0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = np.linalg.norm(inA)*np.linalg.norm(inB)
    return 0.5+0.5*(num/denom)


In [6]:
myMat = np.mat(loadExData())
print(ecludSim(myMat[:,0],myMat[:,4]))
print(ecludSim(myMat[:,0],myMat[:,0]))


0.13367660240019172
1.0


In [7]:
print(cosSim(myMat[:,0],myMat[:,4]))
print(cosSim(myMat[:,0],myMat[:,0]))

0.5472455591261534
0.9999999999999999


In [8]:
print(pearsSim(myMat[:,0],myMat[:,4]))
print(pearsSim(myMat[:,0],myMat[:,0]))

0.23768619407595815
1.0


- 基于物品还是基于用户的相似度，实际上用户数目多，基于用户相似度的复杂度会比较高，反之亦然。因此具体如何使用取决于使用场景，用户数量多则倾向于使用物品相似度；物品多则倾向于用户相似度
- 另外如何评价推荐引擎呢，由于没有预测目标，因此可以采用交叉验证的方法，最后计算预测值和真实打分之间的差异

# e.g. 
## 餐馆菜肴推荐引擎

In [9]:
def standEst(dataMat,user,simMeas,item):
    n = np.shape(dataMat)[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    
    for j in range(n):
        userRating = dataMat[user,j]
        # 寻找两个用户都评分的物品
        if userRating==0:
            continue
        overlap = np.nonzero(np.logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))[0]
        if len(overlap)==0:
            similarity = 0
        else:
            similarity = simMeas(dataMat[overlap,item],dataMat[overlap,j])
            
        # 累计
        simTotal += similarity
        ratSimTotal += similarity*userRating
    
    # 相当于对相似度做了加权归一
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal/simTotal

In [10]:
def recommend(dataMat,user,N=3,simMeas=cosSim,estMethod=standEst):
    unratedItems = np.nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems)==0:
        return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat,user,simMeas,item)
        itemScores.append((item,estimatedScore))
    return sorted(itemScores,key = lambda jj:jj[1],reverse=True)[:N]

In [11]:
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [1, 1, 1, 0, 0],
           [5, 5, 5, 0, 0]]

myMat = np.mat(loadExData())
myMat[0,1]=myMat[0,0]=myMat[1,0]=myMat[2,0]=4
myMat[3,3] = 2
# 行表示用户，列表示物品
myMat

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [1, 1, 1, 0, 0],
        [5, 5, 5, 0, 0]])

In [12]:
print(recommend(myMat,2))
print(recommend(myMat,2,simMeas=ecludSim))
print(recommend(myMat,2,simMeas=pearsSim))

[(2, 2.5), (1, 2.0243290220056256)]
[(2, 3.0), (1, 2.8266504712098603)]
[(2, 2.5), (1, 2.0)]


---
实际数据集要比上述例子更大且更稀疏例如
![一个更大的用户菜肴矩阵，其中很多菜都没有评分](./dataExample1.jpg)

In [13]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]
U,Sigma,VT = np.linalg.svd(np.mat(loadExData2()))
Sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

In [14]:
# 寻找多少奇异值能占据90%的信息
Sig2 = Sigma**2
np.sum(Sig2)*0.9

487.7999999999996

In [15]:
print(np.sum(Sig2[:2]))
print(np.sum(Sig2[:3]))

378.8295595113579
500.5002891275793


---
我们利用SVD将所有菜肴映射到一个低维空间中，在低维空间中利用相似度来进行推荐

In [16]:
def svdEst(dataMat,user,simMeas,item):
    n = np.shape(dataMat)[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    U,Sigma,VT = np.linalg.svd(dataMat)
    Sig4 = np.mat(np.eye(4)*Sigma[:4])
    
    #降维后的物品
    xformedItems = dataMat.T * U[:,:4]*Sig4.I
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating==0 or j==item:
            continue
        similarity = simMeas(xformedItems[item,:].T,xformedItems[j,:].T)
        simTotal += similarity
        ratSimTotal += similarity*userRating
    if simTotal==0:
        return 0
    else:
        return ratSimTotal/simTotal

In [17]:
myMat = np.mat(loadExData2())
recommend(myMat,1,estMethod=svdEst)

[(4, 3.344714938469228), (7, 3.3294020724526967), (9, 3.3281008763900686)]

In [18]:
recommend(myMat,1,estMethod=svdEst,simMeas=pearsSim)

[(4, 3.346952186702173), (9, 3.33537965732747), (6, 3.3071930278130366)]

- 该实例简单展示了推荐引擎的工作流程以及svd将数据映射为重要特征的过程
- SVD实际上不用每次评估都分解，使用前运行一次即可，大型系统中svd每天运行一次或者运行频率不高，并且还要离线运行
- 矩阵表示方法：稀疏矩阵的存储传输也是提高效率的方式
- 每次推荐计算多个物品间相似度也是一大消耗，因此需要时这些物品间得分也可以被另一用户使用；实际中，另一普遍的做法就是离线计算并保存相似度得分
- 推荐引擎面临的另一问题就是如何在缺乏数据时给出推荐——冷启动问题
- 冷启动问题的解决就是将推荐看成搜索问题，内部表现上不同方法有所不同但是对用户是透明的。为此我们需要推荐物品的属性，将这些属性作为相似度的数据计算，这被称为**基于内容的推荐**，该方法效果可能不如协同过滤的好，但这只是个开始


## 基于SVD的图像压缩

In [19]:
def printMat(inMat,thresh=0.8):
    for i in range(32):
        for k in range(32):
            if float(inMat[i,k])>thresh:
                print(1,end='')
            else:
                print(0,end='')
        print('')

In [20]:
def imgCompress(numSV=3,thresh=0.8):
    myl = []
    for line in open('../data/SVD/0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = np.mat(myl)
    print('*'*5,'Original  matrix','*'*5)
    printMat(myMat,thresh)
    
    U,Sigma,VT = np.linalg.svd(myMat)
    SigRecon = np.mat(np.zeros((numSV,numSV)))
    for k in range(numSV):
        SigRecon[k,k] = Sigma[k]
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:]
    
    print('*'*5,'reconstructed  matrix  using %d singular'%numSV,'*'*5)
    printMat(reconMat,thresh)

In [21]:
imgCompress(2)

***** Original  matrix *****
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000000
00000000000111

可见我们使用$U_{32\times 2},V^T_{2\times 32},\Sigma_{2\times 2} $共32*2*2+2=130个像素就能存储1024个原始像素的图片，因此获得了将近10倍的压缩率